In [5]:
from bs4 import BeautifulSoup
import requests
import random
import pandas as pd
import re
import time
import numpy as np
import pickle

In [136]:
user_agent = {'User-agent': 'Mozilla/5.0'}

In [25]:
artists=[]
years=[]
release_countries=[]
labels=[]
artist_links=[]
albums=[]
album_links=[]

In [26]:
for i in range(0,21):
    '''grabs most collected records from discogs'''
    if i==0:
        url = "https://www.discogs.com/search/?sort=have%2Cdesc&ev=em_rs&format_exact=Vinyl&layout=sm"
    if i > 0:
        url = "https://www.discogs.com/search/?sort=have%2Cdesc&ev=em_rs&format_exact=Vinyl&layout=sm&page=" + str(i)
    time.sleep(30+5*random.random())
    response = requests.get(url, headers = user_agent)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    #adding artists to the lists
    for element in soup.find(id='search_results').find_all(class_="card_body"):
        try:
            years.append(element.find_all(class_="card_release_year")[0].text)
        except:
            years.append(np.nan)
        try:
            labels.append(element.find('p', class_="card_info").find_all('a')[0].text)
        except:
            labels.append(np.nan)
        try:
            release_countries.append(element.find_all(class_="card_release_country")[0].text)
        except:
            release_countries.append(np.nan)
        try:
            albums.append(element.find_all("a", class_="search_result_title")[0].text)
        except:
            albums.append(np.nan)
        try:
            album_links.append('https://www.discogs.com'+element.find_all('a', class_="search_result_title")[0].get('href'))
        except:
            album_links.append(np.nan)
        try:
            artists.append(element.find_all('a')[0].text)
        except:
            artists.append(np.nan)
        try:
            artist_links.append('https://www.discogs.com'+element.find_all('a')[0].get('href'))
        except:
            artist_links.append(np.nan)

In [27]:
len(years)

1050

In [28]:
len(albums)

1050

In [29]:
len(artists)

1050

In [30]:
len(artist_links)

1050

In [31]:
len(album_links)

1050

In [32]:
len(labels)

1050

In [33]:
len(release_countries)

1050

In [20]:
top_collected = pd.DataFrame({'artists':artists,'albums':albums, 'artist_links':artist_links, 'album_links':album_links, 'release_year':years, 'label':labels, 'release_country':release_countries})

In [34]:
top_all_collected = pd.DataFrame({'artists':artists,'albums':albums, 'artist_links':artist_links, 'album_links':album_links, 'release_year':years, 'label':labels, 'release_country':release_countries})

In [35]:
top_all_collected.tail()

,artists,albums,artist_links,album_links,release_year,label,release_country
1045,The Beatles,The Beatles At The Hollywood Bowl,https://www.discogs.com/artist/82730-The-Beatles,https://www.discogs.com/The-Beatles-The-Beatle...,1977,Parlophone,UK
1046,Fine Young Cannibals,The Raw & The Cooked,https://www.discogs.com/artist/4711-Fine-Young...,https://www.discogs.com/Fine-Young-Cannibals-T...,1988,London Records,UK
1047,Elton John,A Single Man,https://www.discogs.com/artist/57103-Elton-John,https://www.discogs.com/Elton-John-A-Single-Ma...,1978,The Rocket Record Company,UK
1048,David Bowie,Never Let Me Down,https://www.discogs.com/artist/10263-David-Bowie,https://www.discogs.com/David-Bowie-Never-Let-...,1987,EMI America,UK
1049,Daryl Hall + John Oates,H2O,https://www.discogs.com/artist/95886-Daryl-Hal...,https://www.discogs.com/Daryl-Hall-John-Oates-...,1982,RCA,US


In [58]:
top_all_collected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   artists          1050 non-null   object
 1   albums           1050 non-null   object
 2   artist_links     1050 non-null   object
 3   album_links      1050 non-null   object
 4   release_year     1047 non-null   object
 5   label            1050 non-null   object
 6   release_country  1049 non-null   object
dtypes: object(7)
memory usage: 57.5+ KB


In [36]:
import pickle

In [38]:
with open('top_1050_collected_records.pickle', 'wb') as save_file:
    pickle.dump(top_all_collected, save_file)

In [39]:
import os
os.getcwd()

'/Users/beth/Desktop'

In [42]:
from datetime import datetime

In [49]:
first_release_links = []
number_of_songs= []
average_song_length= []

In [50]:
for url in top_all_collected['album_links']:
    #from album main page
    time.sleep(10+2*random.random())
    response = requests.get(url, headers = user_agent)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    #link to album 1st release page
    try:
        first_release_links.append('https://www.discogs.com' + soup.find('table', id='versions').find('td', class_='title').find_all('a')[0].get('href'))
    except:
        first_release_links.append(np.nan)
    
    #number of songs
    s=0
    try:
        for element in soup.find('table', class_="playlist").find_all('tr', class_="tracklist_track track"):
            for song in element.find('span', class_="tracklist_track_title"):
                s += 1
        number_of_songs.append(s)
    except:
        number_of_songs.append(np.nan)
    
    #avg song length
    form='%M:%S'
    song_lengths=[]
    try:
        for element in soup.find('table', class_="playlist").find_all('tr', class_="tracklist_track track"):
            t = element.find('td', class_="tracklist_track_duration").find('span').text
            t = datetime.strptime(t,form)
            song_lengths.append(t)
        avg_time=datetime.strftime(datetime.fromtimestamp(sum(map(datetime.timestamp,song_lengths))/len(song_lengths)),"%M:%S")
        average_song_length.append(avg_time)
    except:
        average_song_length.append(np.nan)
        
        

In [51]:
album_main_pages = pd.DataFrame({'first_release_links':first_release_links,'number_of_songs':number_of_songs,'average_song_length':average_song_length})

In [56]:
album_main_pages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   first_release_links  1050 non-null   object
 1   number_of_songs      1050 non-null   int64 
 2   average_song_length  831 non-null    object
dtypes: int64(1), object(2)
memory usage: 24.7+ KB


In [54]:
with open('album_main_pages.pickle', 'wb') as save_file:
    pickle.dump(album_main_pages, save_file)

In [132]:
with open('top_1050_collected_records.pickle', 'rb') as read_file:
    top_1050_collected_records = pickle.load(read_file)

In [133]:
top_1050_collected_records['artist_links'][0]

'https://www.discogs.com/artist/45467-Pink-Floyd'

In [150]:
total_artist_albums = []
artist_first_years = []
artist_last_years = []

In [151]:
for url in top_1050_collected_records['artist_links']:
    #get info from artist pages
    response = requests.get(url, headers = user_agent)
    time.sleep(5+1*random.random())
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    #total albums
    try:
        total_artist_albums.append(soup.find(id="discography_wrapper").find_all('span', class_='facet_count')[1].text)
    except:
        total_artist_albums.append(np.nan)
        
    #first album year
    try:
        artist_first_years.append(soup.find('table', id="artist").find_all('td', class_="year has_header")[0].text)
    except:
        artist_first_years.append(np.nan)
        
    #last album year
    try:
        url=url + "?sort=year%2Cdesc&limit=25"
        time.sleep(5+1*random.random())
        response = requests.get(url, headers = user_agent)
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        artist_last_years.append(soup.find('table', id="artist").find_all('td', class_="year has_header")[0].text)
    except:
        artist_last_years.append(np.nan)
    

In [154]:
artist_data = pd.DataFrame({'total_artist_albums':total_artist_albums,'artist_last_years':artist_last_years,'artist_first_years':artist_first_years})

In [159]:
with open('artist_data.pickle', 'wb') as write_file:
    pickle.dump(artist_data, write_file)

In [161]:
with open('artist_data.pickle', 'rb') as read_file:
    test = pickle.load(read_file)

In [163]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   total_artist_albums  1039 non-null   object
 1   artist_last_years    1039 non-null   object
 2   artist_first_years   1039 non-null   object
dtypes: object(3)
memory usage: 24.7+ KB


In [6]:
with open('album_main_pages.pickle', 'rb') as read_file:
    album_main_pages = pickle.load(read_file)

In [10]:
album_main_pages.head()
#might need number of songs + 1

,first_release_links,number_of_songs,average_song_length
0,https://www.discogs.com/Pink-Floyd-The-Dark-Si...,9,NaN
1,https://www.discogs.com/The-Beatles-Sgt-Pepper...,12,03:04
2,https://www.discogs.com/The-Beatles-Abbey-Road...,16,02:41
3,https://www.discogs.com/Pink-Floyd-Wish-You-We...,3,NaN
4,https://www.discogs.com/Michael-Jackson-Thrill...,8,04:31


In [44]:
users_have = []
users_want = []
user_rating = []
lowest_price = []
median_price = []
highest_price = []
last_sold = []
number_for_sale = []
styles = []
genres = []
versions = []

In [45]:
for url in album_main_pages['first_release_links']:
    #get info from the first release of an album page
    response = requests.get(url, headers = user_agent)
    time.sleep(5+1*random.random())
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    #getting info for the record stats, including price
    stats =[element for element in soup.find(id = "statistics").find_all('li')]
    try:
        users_have.append(re.search(':\n([0-9]*)',stats[0].text).group(1))
    except:
        users_have.append(np.nan)
    try:
        users_want.append(re.search(':\n([0-9]*)',stats[1].text).group(1))
    except:
        users_want.append(np.nan)
    try:
        user_rating.append(re.search(':\n([0-9].[0-9]{1,2})', stats[2].text).group(1))
    except:
        user_rating.append(np.nan)
    try:
        lowest_price.append(re.search('\$([0-9,]*.[0-9]{2})',stats[5].text).group(1))
    except:
        lowest_price.append(np.nan)
    try:
        median_price.append(re.search('\$([0-9,]*.[0-9]{2})',stats[6].text).group(1))
    except:
        median_price.append(np.nan)
    try:
        highest_price.append(re.search('\$([0-9,]*.[0-9]{2})',stats[7].text).group(1))
    except:
        highest_price.append(np.nan)
    try:
        last_sold.append(re.search(':\n([0-9]{2} [A-z]{3} [0-9]{2})',stats[4].text).group(1))
    except:
        last_sold.append(np.nan)
        
    #the number of versions
    try:
        versions.append(re.search('of ([0-9]*)\)',soup.find('h3',{'data-for':'.m_versions'}).text).group(1))
    except:
        versions.append(np.nan)
        
    #getting for sale info from marketplace section
    try:
        element = soup.find(class_ = "marketplace_for_sale_count").find('strong').text
        number_for_sale.append(re.search('([0-9,]*)',element).group())
    except:
        number_for_sale.append(np.nan)
        
    #find genre and styles (may be multiple, so a list)
    try:
        all_styles=[]
        for link in soup.find(class_="profile").find_all("a"):
            if 'style' in link.get('href'):
                all_styles.append(link.text)
        styles.append(all_styles)
    except:
        styles.append(np.nan)
        
    try:
        all_genres=[]
        for link in soup.find(class_="profile").find_all("a"):
            if 'genre' in link.get('href'):
                all_genres.append(link.text)
        genres.append(all_genres)
    except:
        genres.append(np.nan)
    

AttributeError: 'NoneType' object has no attribute 'find_all'

In [46]:
len(users_have)

46

In [47]:
album_main_pages['first_release_links'][46]

'https://www.discogs.com/artist/39766-The-Velvet-Underground'

In [50]:
album_main_pages.iloc[46]

first_release_links    https://www.discogs.com/artist/39766-The-Velve...
number_of_songs                                                       10
average_song_length                                                04:29
Name: 46, dtype: object

In [57]:
for url in album_main_pages['first_release_links'][46:]:
    #get info from the first release of an album page
    response = requests.get(url, headers = user_agent)
    time.sleep(5+1*random.random())
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    #getting info for the record stats, including price
    try:
        stats =[element for element in soup.find(id = "statistics").find_all('li')]
    except:
        users_have.append(np.nan)
        users_want.append(np.nan)
        user_rating.append(np.nan)
        lowest_price.append(np.nan)
        median_price.append(np.nan)
        highest_price.append(np.nan)
        last_sold.append(np.nan)
        #I should have put stats = [] here, my run resulted in an extra row after each album with no stats
    try:
        users_have.append(re.search(':\n([0-9]*)',stats[0].text).group(1))
    except:
        users_have.append(np.nan)
    try:
        users_want.append(re.search(':\n([0-9]*)',stats[1].text).group(1))
    except:
        users_want.append(np.nan)
    try:
        user_rating.append(re.search(':\n([0-9].[0-9]{1,2})', stats[2].text).group(1))
    except:
        user_rating.append(np.nan)
    try:
        lowest_price.append(re.search('\$([0-9,]*.[0-9]{2})',stats[5].text).group(1))
    except:
        lowest_price.append(np.nan)
    try:
        median_price.append(re.search('\$([0-9,]*.[0-9]{2})',stats[6].text).group(1))
    except:
        median_price.append(np.nan)
    try:
        highest_price.append(re.search('\$([0-9,]*.[0-9]{2})',stats[7].text).group(1))
    except:
        highest_price.append(np.nan)
    try:
        last_sold.append(re.search(':\n([0-9]{2} [A-z]{3} [0-9]{2})',stats[4].text).group(1))
    except:
        last_sold.append(np.nan)
        
    #the number of versions
    try:
        versions.append(re.search('of ([0-9]*)\)',soup.find('h3',{'data-for':'.m_versions'}).text).group(1))
    except:
        versions.append(np.nan)
        
    #getting for sale info from marketplace section
    try:
        element = soup.find(class_ = "marketplace_for_sale_count").find('strong').text
        number_for_sale.append(re.search('([0-9,]*)',element).group())
    except:
        number_for_sale.append(np.nan)
        
    #find genre and styles (may be multiple, so a list)
    try:
        all_styles=[]
        for link in soup.find(class_="profile").find_all("a"):
            if 'style' in link.get('href'):
                all_styles.append(link.text)
        styles.append(all_styles)
    except:
        styles.append(np.nan)
        
    try:
        all_genres=[]
        for link in soup.find(class_="profile").find_all("a"):
            if 'genre' in link.get('href'):
                all_genres.append(link.text)
        genres.append(all_genres)
    except:
        genres.append(np.nan)
    

In [58]:
first_version_page_df = pd.DataFrame({'users_have':users_have,'users_want':users_want,'user_rating':user_rating,'lowest_price':lowest_price,'median_price':median_price,'highest_price':highest_price,'last_sold':last_sold,'number_for_sale':number_for_sale,'styles':styles,'genres':genres,'versions':versions})

ValueError: arrays must all be same length

In [59]:
len(users_have)

1068

In [60]:
len(users_want)

1068

In [61]:
len(user_rating)

1068

In [62]:
len(lowest_price)

1068

In [63]:
album_main_pages.shape

(1050, 3)

In [64]:
len(lowest_price)

1068

In [65]:
len(highest_price)

1068

In [66]:
len(median_price)

1068

In [67]:
len(styles)

1050

In [68]:
len(genres)

1050

In [69]:
len(number_for_sale)

1050

In [71]:
len(versions)

1050

In [72]:
df_B = pd.DataFrame({'styles':styles,'genres':genres,'number_for_sale':number_for_sale, 'versions':versions})

In [73]:
df_B.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   styles           1049 non-null   object
 1   genres           1049 non-null   object
 2   number_for_sale  1034 non-null   object
 3   versions         1033 non-null   object
dtypes: object(4)
memory usage: 32.9+ KB


In [74]:
df_A = pd.DataFrame({'users_have':users_have,'users_want':users_want,'user_rating':user_rating,'lowest_price':lowest_price,'median_price':median_price,'highest_price':highest_price,'last_sold':last_sold})

In [79]:
df_A.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   users_have     1050 non-null   object
 1   users_want     1050 non-null   object
 2   user_rating    1048 non-null   object
 3   lowest_price   1045 non-null   object
 4   median_price   1045 non-null   object
 5   highest_price  1045 non-null   object
 6   last_sold      1045 non-null   object
dtypes: object(7)
memory usage: 58.5+ KB


In [81]:
df_A[df_A['users_have'].isnull()==True]

,users_have,users_want,user_rating,lowest_price,median_price,highest_price,last_sold
46,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,NaN,NaN,NaN,NaN,NaN,NaN,NaN
333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
343,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,NaN,NaN,NaN,NaN,NaN,NaN,NaN
426,NaN,NaN,NaN,NaN,NaN,NaN,NaN
561,NaN,NaN,NaN,NaN,NaN,NaN,NaN
571,NaN,NaN,NaN,NaN,NaN,NaN,NaN
600,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
df_A.iloc[40:50]

,users_have,users_want,user_rating,lowest_price,median_price,highest_price,last_sold
40,18941,2177,4.24,0.99,4.50,20.00,19 Sep 20
41,1799,3478,4.55,32.47,97.73,168.83,04 Sep 20
42,8877,2590,4.49,2.53,12.99,29.99,31 Aug 20
43,6987,2854,4.42,15.58,25.96,64.94,03 Sep 20
44,8062,2159,4.38,6.00,14.27,29.00,09 Sep 20
45,1781,1941,4.26,32.93,104.89,199.00,24 Aug 20
46,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,1781,1941,4.26,32.93,104.89,199.00,24 Aug 20
48,9224,2420,4.45,9.74,12.99,42.86,01 Sep 20
49,3104,2656,4.41,25.96,199.99,294.12,07 Aug 20


In [90]:
df_A.iloc[200:210]

,users_have,users_want,user_rating,lowest_price,median_price,highest_price,last_sold
200,3991,1121,4.13,12.97,25.32,78.71,05 Sep 20
201,4199,1392,4.08,6.49,12.99,29.86,06 Sep 20
202,686,663,4.48,20.00,37.50,99.99,26 Jul 20
203,11533,2731,4.5,6.99,18.88,99.99,07 Sep 20
204,1224,3533,4.32,38.90,338.80,924.46,29 Jun 20
205,6271,6390,4.67,30.00,75.00,"3,000.00",06 Aug 20
206,7199,276,3.97,2.94,5.33,9.08,22 Aug 20
207,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,7199,276,3.97,2.94,5.33,9.08,22 Aug 20
209,1696,1147,4.1,25.88,34.56,63.64,17 Aug 20


In [94]:
duplicates_after_NAs = list(map(lambda x: x+1, list(df_A[df_A['users_have'].isnull()==True].index)))

In [95]:
duplicates_after_NAs 

[47,
 98,
 208,
 334,
 344,
 358,
 427,
 562,
 572,
 601,
 616,
 719,
 746,
 785,
 837,
 944,
 1008,
 1060]

In [96]:
len(duplicates_after_NAs)

18

In [97]:
df_A = df_A.drop(duplicates_after_NAs, axis = 0)

In [98]:
df_A.shape

(1050, 7)

In [103]:
first_edition_album_info = pd.concat([df_A, df_B],axis=1)

In [104]:
first_edition_album_info.shape

(1067, 11)

In [105]:
df_A.reset_index(inplace=True)

In [106]:
first_edition_album_info = pd.concat([df_A, df_B],axis=1)

In [107]:
first_edition_album_info.shape

(1050, 12)

In [115]:
with open('first_edition_album_info.pickle', 'wb') as write_file:
    pickle.dump(first_edition_album_info, write_file)
    

In [112]:
first_edition_album_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            1050 non-null   int64 
 1   users_have       1032 non-null   object
 2   users_want       1032 non-null   object
 3   user_rating      1030 non-null   object
 4   lowest_price     1027 non-null   object
 5   median_price     1027 non-null   object
 6   highest_price    1027 non-null   object
 7   last_sold        1027 non-null   object
 8   styles           1049 non-null   object
 9   genres           1049 non-null   object
 10  number_for_sale  1034 non-null   object
 11  versions         1033 non-null   object
dtypes: int64(1), object(11)
memory usage: 98.6+ KB


In [110]:
del df_A

In [111]:
del df_B

In [116]:
with open('first_edition_album_info.pickle', 'rb') as read_file:
    test = pickle.load(read_file)

In [117]:
test.head()

,index,users_have,users_want,user_rating,lowest_price,median_price,highest_price,last_sold,styles,genres,number_for_sale,versions
0,0,11420,7607,4.67,88.18,246.75,"1,381.05",10 Aug 20,"[Prog Rock, Psychedelic Rock]",[Rock],68,998
1,1,10456,4967,4.55,25.97,110.36,324.61,01 Sep 20,"[Rock & Roll, Psychedelic Rock]",[Rock],150,918
2,2,3636,2394,4.61,19.48,77.65,391.30,29 Sep 20,[Pop Rock],[Rock],28,720
3,3,17723,7965,4.63,23.00,49.68,103.90,29 Aug 20,[Prog Rock],[Rock],130,729
4,4,29276,6393,4.5,4.90,13.24,75.00,18 Sep 20,"[Disco, Soul]","[Funk / Soul, Pop]",197,552


In [118]:
del test

In [123]:
first_edition_album_info['median_price'].sort_values()

284     1,166.00
905         1.29
423         1.30
734         1.81
410        10.00
          ...   
822          NaN
855          NaN
928          NaN
991          NaN
1042         NaN
Name: median_price, Length: 1050, dtype: object

In [124]:
first_edition_album_info.iloc[284]

index                     287
users_have                179
users_want               1175
user_rating              4.44
lowest_price           532.00
median_price         1,166.00
highest_price        1,800.00
last_sold           08 Mar 20
styles             [Hard Bop]
genres                 [Jazz]
number_for_sale           NaN
versions                  227
Name: 284, dtype: object

In [125]:
del first_edition_album_info

In [128]:
album_main_pages['number_of_songs']=album_main_pages['number_of_songs']+1

In [129]:
album_main_pages.head()

,first_release_links,number_of_songs,average_song_length
0,https://www.discogs.com/Pink-Floyd-The-Dark-Si...,10,NaN
1,https://www.discogs.com/The-Beatles-Sgt-Pepper...,13,03:04
2,https://www.discogs.com/The-Beatles-Abbey-Road...,17,02:41
3,https://www.discogs.com/Pink-Floyd-Wish-You-We...,4,NaN
4,https://www.discogs.com/Michael-Jackson-Thrill...,9,04:31


In [130]:
with open('album_main_pages_2.pickle', 'wb') as write_file:
    pickle.dump(album_main_pages, write_file)

In [131]:
del album_main_pages

In [152]:
top_1050_collected_records['artist']

KeyError: 'artist'